# F# によるデータ解析・機械学習のデモ

この notebook は kaggle のチュートリアルコンペティションである [Titanic](https://www.kaggle.com/competitions/titanic) のデータを F# (w/ Polyglot Notebook)を用いて解析するデモンストレーションです．

以下の構成でファイルが格納されていることを想定しています：

- data/ : データファイル．[kaggle からダウンロードする](https://www.kaggle.com/competitions/titanic)．
  - train.csv : 学習用データ．
  - test.csv : テスト用データ
- src/ : スクリプト．
  - main.ipynb : このファイル．
  - Utils.fsx : ユーティリティ関数を定義したファイル．
- submission/ : 提出用ファイル．


依存パッケージを読み込みます．
また，追加のユーティリティを定義したファイル `Utils.fsx` をロードしておきます．


In [1]:
#r "nuget: Deedle"
#r "nuget: Accord.MachineLearning"
#r "nuget: Accord.Statistics"

Installed Packages Accord.MachineLearning, 3.8.0 Accord.Statistics, 3.8.0 Deedle, 3.0.0

In [2]:
#load "Utils.fsx"

open Utils

## Deedle ... データフレームライブラリ

最初に，データフレームライブラリ [Deedle](https://bluemountaincapital.github.io/Deedle/) を用いてタイタニック号の乗客データを読み込みます．

### データの読み込み

`Deedle.Frame` はデータフレームを格納する型です．配置したトレーニングファイル，テストファイルを `Frame.ReadCsv` 関数で読み込み，`Frame.indexRowsInt` 関数で `PassengerId` を行インデックスに設定します．


In [3]:
open Deedle

let train =
    Frame.ReadCsv(__SOURCE_DIRECTORY__ + "/../data/titanic/train.csv") // トレーニングデータを読み込み，
    |> Frame.indexRowsInt "PassengerId" // PassengerId を行名に設定する．

let test =
    Frame.ReadCsv(__SOURCE_DIRECTORY__ + "/../data/titanic/test.csv") // テストデータを読み込み，
    |> Frame.indexRowsInt "PassengerId" // PassengerId を行名に設定する．

// テストデータの正答率をローカルで調べたい場合は代わりに以下のコードで `train` を分割して読み込む．

// let titanic =
//     Frame.ReadCsv(__SOURCE_DIRECTORY__ + "/../data/titanic/train.csv") |> Frame.indexRowsInt "PassengerId"

// let train = titanic.GetRowsAt([|0..790|])
// let test = titanic.GetRowsAt([|791..890|])

In [4]:
// トレーニングデータを表示する．
train.Print()

       Survived Pclass Name                                                Sex    Age       SibSp Parch Ticket           Fare    Cabin Embarked 
1   -> False    3      Braund, Mr. Owen Harris                             male   22        1     0     A/5 21171        7.25          S        
2   -> True     1      Cumings, Mrs. John Bradley (Florence Briggs Thayer) female 38        1     0     PC 17599         71.2833 C85   C        
3   -> True     3      Heikkinen, Miss. Laina                              female 26        0     0     STON/O2. 3101282 7.925         S        
4   -> True     1      Futrelle, Mrs. Jacques Heath (Lily May Peel)        female 35        1     0     113803           53.1    C123  S        
5   -> False    3      Allen, Mr. William Henry                            male   35        0     0     373450           8.05          S        
6   -> False    3      Moran, Mr. James                                    male   <missing> 0     0     330877           8.4583   

In [5]:
// テストデータを表示する．
test.Print()

        Pclass Name                                                    Sex    Age       SibSp Parch Ticket             Fare    Cabin Embarked 
892  -> 3      Kelly, Mr. James                                        male   34.5      0     0     330911             7.8292        Q        
893  -> 3      Wilkes, Mrs. James (Ellen Needs)                        female 47        1     0     363272             7             S        
894  -> 2      Myles, Mr. Thomas Francis                               male   62        0     0     240276             9.6875        Q        
895  -> 3      Wirz, Mr. Albert                                        male   27        0     0     315154             8.6625        S        
896  -> 3      Hirvonen, Mrs. Alexander (Helga E Lindqvist)            female 22        1     1     3101298            12.2875       S        
897  -> 3      Svensson, Mr. Johan Cervin                              male   14        0     0     7538               9.225         S        

今回のデータフレームは `Frame<int, string>` という型を持ちます．これは行のインデックスが `int` 型，列のインデックスが `string` 型であることを意味します．
データフレームの値の型は，整数であったり文字列であったりするので，特に指定しない限り動的型 `obj` として扱われます．
たとえば `string` 型であることが分かっているセルの値 `v` を `string` 型として扱いたい場合は，`v :?> string` というダウンキャストを行います．


### データクレンジング・データの前処理

上で表示した通り，データフレームのいくつかの列には欠損値が含まれています．データ解析の前にこれらを埋める必要があります．また，整数や boolean，文字列であるような列の値を変換し，浮動小数点型のみを含むデータフレームに変換します．

Deedle のデータフレームでは，欠損したセルの値をピンポイントで読み込もうとしたタイミングで例外が発生します．また，`Series` を読み込む関数，たとえば `Series.values` では欠損値はスキップされ，値の個数が減少した列が返される点に注意が必要です．欠損値を `None`，欠損値でない値を `Some` として取り出すには，たとえば `Series.valuesAll` のように末尾に `All` を付けた関数を用います．

まずは欠損値の数を表示する関数 `printCleansingInfo` を作成しましょう．なお，データ上は `Embarked` や `Cabin` なども欠損値を含みますが，文字列型の列については欠損値は空文字列として取り込まれているので，これらは欠損値として扱わないことにします．


In [6]:
let printCleansingInfo (frame: Frame<'R, 'C>) =
    let missingValueCounts =
        frame.Columns
        |> Series.observations
        |> Seq.map (
            fun (key, col) ->
                let length = col.ValuesAll |> Seq.length // 欠損値も含めた列の長さ．
                let valueCount = col.ValueCount // 欠損値を除いた列の長さ．
                let missingCount = length - valueCount // 欠損値の数．
                key, missingCount
        )
        |> Seq.filter (fun (key, missingCount) -> missingCount > 0)
    
    missingValueCounts
    |> Seq.iter (fun (key, count) -> printfn "Column %A has %d missing values" key count)

    if missingValueCounts |> Seq.isEmpty then
        printfn "No missing values"

printfn "Train data:"
printCleansingInfo train

printfn "Test data:"
printCleansingInfo test

Train data:
Column "Age" has 177 missing values
Test data:
Column "Age" has 86 missing values
Column "Fare" has 1 missing values


これらの欠損値がなくなり，すべてのセルの値が `float` 型になるよう，データフレームを変換します．
以下では `Utils.fsx` 内で定義した関数を用いて各列を変換していきます．


In [7]:
let preprocess (frame: Frame<int, string>) =
    frame
    // Pclass は int から float に変換する．
    |> FrameExt.modifyCol "Pclass" float
    // Name は削除する．
    |> Frame.dropCol "Name"
    // Sex は one-hot エンコーディングを行う．
    // カテゴリは `male` か `female` であるので，`male` であるかどうかを示す列を追加する．
    |> FrameExt.oneHot "Sex" [ "male" ]
    // Age は欠損値を平均値で置き換え，そのうえで欠損値であるか示す列を追加する．
    |> FrameExt.addColMissingTag "Age"
    |> FrameExt.modifyCol "Age_missing" boolToFloat
    |> FrameExt.replaceColMissing "Age" (frame?Age |> Stats.mean)
    // SibSp，Parch はそれぞれ int から float に変換する．
    |> FrameExt.modifyCol "SibSp" float
    |> FrameExt.modifyCol "Parch" float
    // Ticket は削除する．
    |> Frame.dropCol "Ticket"
    // Fare は欠損値を平均値で置き換える．欠損値であるか示す列は，欠損値がわずかであるため省略する．
    |> FrameExt.modifyCol "Fare" (fun value -> value.ToString() |> float)
    |> FrameExt.replaceColMissing "Fare" (frame?Fare |> Stats.mean)
    // Cabin は削除する．
    |> Frame.dropCol "Cabin"
    // Embarked は one-hot エンコーディングを行う．
    // カテゴリには `S`，`Q`，`C` で，さらに欠損値を表す空文字列があるので，`S` であるかどうか，`Q` であるかどうか，`C` であるかどうかを示す列を追加する．
    |> FrameExt.oneHot "Embarked" [ "S"; "Q"; "C" ]

In [8]:
// トレーニングデータを前処理する．
let train' = preprocess train |> FrameExt.modifyCol "Survived" boolToFloat

train'.Print()

       Pclass Sex_male Age_missing Age               SibSp Parch Fare    Embarked_C Embarked_Q Embarked_S Survived 
1   -> 3      1        0           22                1     0     7.25    0          0          1          0        
2   -> 1      0        0           38                1     0     71.2833 1          0          0          1        
3   -> 3      0        0           26                0     0     7.925   0          0          1          1        
4   -> 1      0        0           35                1     0     53.1    0          0          1          1        
5   -> 3      1        0           35                0     0     8.05    0          0          1          0        
6   -> 3      1        1           29.69911764705882 0     0     8.4583  0          1          0          0        
7   -> 1      1        0           54                0     0     51.8625 0          0          1          0        
8   -> 3      1        0           2                 3     1     21.075 

In [9]:
// テストデータを前処理する．
let test' = 
    if test.ColumnKeys |> Seq.contains "Survived" then
        // テストデータに Survived 列がある場合は，それを float に変換する．
        preprocess test |> FrameExt.modifyCol "Survived" Utils.boolToFloat
    else
        preprocess test

test'.Print()

        Pclass Sex_male Age_missing Age                SibSp Parch Fare    Embarked_C Embarked_Q Embarked_S 
892  -> 3      1        0           34.5               0     0     7.8292  0          1          0          
893  -> 3      0        0           47                 1     0     7       0          0          1          
894  -> 2      1        0           62                 0     0     9.6875  0          1          0          
895  -> 3      1        0           27                 0     0     8.6625  0          0          1          
896  -> 3      0        0           22                 1     1     12.2875 0          0          1          
897  -> 3      1        0           14                 0     0     9.225   0          0          1          
898  -> 3      0        0           30                 0     0     7.6292  0          1          0          
899  -> 2      1        0           26                 1     1     29      0          0          1          
900  -> 3      0   

In [10]:
// 欠損値を改めて確認する．
printCleansingInfo train'
printCleansingInfo test'

No missing values
No missing values


## Accord.NET ... 機械学習ライブラリ

続いて [Accord.NET](http://accord-framework.net/) を用いてデータの学習を行います．今回は多変量ロジスティック回帰分析とランダムフォレストをデモンストレートします．

まずはデータフレームから予測変数と目的変数の配列を取り出します．


In [11]:
let inputs: float array array =
    train'
    |> Frame.dropCol "Survived"
    |> Frame.toJaggedArray

let survived: int array =
    train'
    |> Frame.getCol "Survived"
    |> Series.values
    |> Seq.toArray

次に，ロジスティック回帰とランダムフォレストのモデルを作成し，学習を行います．

それぞれのモデルは `Accord.Statistics.Analysis.MultivariateLogisticRegression` および `Accord.MachineLearning.DecisionTrees.Learning.C45Learning` に存在します(ランダムフォレストのアルゴリズムはいくつか選択することができます)．モデルを作成し，`.Learn(inputs, survived)` でトレーニングデータを学習させます．


In [12]:
open Accord
open Accord.Statistics.Analysis
open Accord.MachineLearning.DecisionTrees.Learning

// Multinomial logistic regression.
let MLR = MultinomialLogisticRegressionAnalysis().Learn(inputs, survived)

// Random forest.
let RF = C45Learning().Learn(inputs, survived)

// 決定関数の配列を定義．
let deciders: (string * (float array -> int)) array = [|
    "MLR", MLR.Decide
    "RF", RF.Decide
|]

決定関数 `decide: float array -> int` を用いてテストデータの生存者を予測する関数を定義します．


In [13]:
let predictions (decide: float array -> int) =
    test'
    |> Frame.rows
    |> Series.observations
    |> Seq.map (
        fun (id, row) ->
            id,
            row.Values
            |> Seq.map Convert.ToDouble
            |> Seq.toArray
            |> decide
    )
    |> Seq.map (
        fun (id, pred) -> {|
            PassengerId = id
            Survived = pred
        |}
    )

テストデータが `Survived` を含むなら，以下の関数で正答率を計算できます．


In [ ]:
// 正答率の計算
let accuracy (predictions: {| PassengerId: int; Survived: int |} seq) =
    let actual =
        test'
        |> Frame.getCol "Survived"
        |> Series.values
        |> Seq.map (int: float -> int)

    let correctCount =
        predictions
        |> Seq.map (fun p -> p.Survived)
        |> Seq.zip actual
        |> Seq.filter (fun (pred, act) -> pred = act)
        |> Seq.length
    
    float correctCount / float (Seq.length actual)

kaggle への提出用の csv ファイルを作成する関数を定義します．


In [ ]:
// Export csv
let export (fileName: string) (predictions: {| PassengerId: int; Survived: int |} seq) =
    let header = "PassengerId,Survived\n"
    let body =
        predictions
        |> Seq.map (fun p -> sprintf "%d,%d" p.PassengerId p.Survived)
        |> String.concat "\n"
    
    let out = header + body
    File.WriteAllText (__SOURCE_DIRECTORY__ + "/../submission/titanic/" + fileName + ".csv", out)

以上の関数を，ロジスティック回帰とランダムフォレストのそれぞれについて実行します．


In [ ]:
deciders |> Array.iter (fun (name, decide) ->
    printfn "Decider: %s" name

    let predictions = predictions decide

    predictions
    |> Seq.take 10
    |> Seq.iter (fun p -> printfn "PassengerId: %d, Survived: %d" p.PassengerId p.Survived)

    printfn "..."

    if test'.ColumnKeys |> Seq.contains "Survived" then
        predictions |> accuracy |> printfn "Accuracy: %f"
    
    predictions |> export (name + "-submission")
)

Decider: MLR
PassengerId: 892, Survived: 0
PassengerId: 893, Survived: 0
PassengerId: 894, Survived: 0
PassengerId: 895, Survived: 0
PassengerId: 896, Survived: 1
PassengerId: 897, Survived: 0
PassengerId: 898, Survived: 1
PassengerId: 899, Survived: 0
PassengerId: 900, Survived: 1
PassengerId: 901, Survived: 0
...
Decider: RF
PassengerId: 892, Survived: 0
PassengerId: 893, Survived: 0
PassengerId: 894, Survived: 0
PassengerId: 895, Survived: 0
PassengerId: 896, Survived: 1
PassengerId: 897, Survived: 0
PassengerId: 898, Survived: 1
PassengerId: 899, Survived: 0
PassengerId: 900, Survived: 1
PassengerId: 901, Survived: 0
...


こちらで得られた予測結果では，多変量ロジスティック回帰分析の正答率は 0.76794，ランダムフォレストの正答率は 0.77511 となりました．

なお，性別のみを判断基準にした場合の正答率は 0.76555 となるようです．今回は特徴了エンジニアリングなどを特に行っていませんが，データの前処理の際にグルーピングなどの処理を挿入すればさらに精度が向上するものと思われます．
